<a href="https://colab.research.google.com/github/Rohanrathod7/my-ml-labs/blob/main/11_Preprocessing_for_Machine_Learning_in_Python/04_Selecting_Features_for_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 4. Selecting Features for Modeling

In [4]:

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime as dt
# Import confusion matrix and train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LogisticRegression, LinearRegression
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier



url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/11_Preprocessing_for_Machine_Learning_in_Python/dataset/volunteer_opportunities.csv"
# Read the CSV file
# The original code tried to read a feather file as a CSV, and had a UnicodeDecodeError.
# The file extension is feather, so it should be read using pd.read_feather.
# Also, the variable name was confusing, it should be spotify_population.
volunteer = pd.read_csv(url)
display(volunteer.head())


url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/11_Preprocessing_for_Machine_Learning_in_Python/dataset/hiking.json"

# Read the JSON file properly
hiking = pd.read_json(url)

# Display first few rows
display(hiking.head())

url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/11_Preprocessing_for_Machine_Learning_in_Python/dataset/wine_types.csv"
# Read the CSV file
# The original code tried to read a feather file as a CSV, and had a UnicodeDecodeError.
# The file extension is feather, so it should be read using pd.read_feather.
# Also, the variable name was confusing, it should be spotify_population.
wine = pd.read_csv(url)
display(wine.head())

,opportunity_id,content_id,vol_requests,event_time,title,hits,summary,is_priority,category_id,category_desc,...,end_date_date,status,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,4996,37004,50,0,Volunteers Needed For Rise Up & Stay Put! Home...,737,Building on successful events last summer and ...,NaN,NaN,NaN,...,July 30 2011,approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5008,37036,2,0,Web designer,22,Build a website for an Afghan business,NaN,1.0,Strengthening Communities,...,February 01 2011,approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5016,37143,20,0,Urban Adventures - Ice Skating at Lasker Rink,62,Please join us and the students from Mott Hall...,NaN,1.0,Strengthening Communities,...,January 29 2011,approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5022,37237,500,0,Fight global hunger and support women farmers ...,14,The Oxfam Action Corps is a group of dedicated...,NaN,1.0,Strengthening Communities,...,March 31 2012,approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5055,37425,15,0,Stop 'N' Swap,31,Stop 'N' Swap reduces NYC's waste by finding n...,NaN,4.0,Environment,...,February 05 2011,approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Prop_ID,Name,Location,Park_Name,Length,Difficulty,Other_Details,Accessible,Limited_Access,lat,lon
0,B057,Salt Marsh Nature Trail,"Enter behind the Salt Marsh Nature Center, loc...",Marine Park,0.8 miles,None,<p>The first half of this mile-long trail foll...,Y,N,NaN,NaN
1,B073,Lullwater,Enter Park at Lincoln Road and Ocean Avenue en...,Prospect Park,1.0 mile,Easy,Explore the Lullwater to see how nature thrive...,N,N,NaN,NaN
2,B073,Midwood,Enter Park at Lincoln Road and Ocean Avenue en...,Prospect Park,0.75 miles,Easy,Step back in time with a walk through Brooklyn...,N,N,NaN,NaN
3,B073,Peninsula,Enter Park at Lincoln Road and Ocean Avenue en...,Prospect Park,0.5 miles,Easy,Discover how the Peninsula has changed over th...,N,N,NaN,NaN
4,B073,Waterfall,Enter Park at Lincoln Road and Ocean Avenue en...,Prospect Park,0.5 miles,Easy,Trace the source of the Lake on the Waterfall ...,N,N,NaN,NaN


,Type,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


**Selecting relevant features**  
In this exercise, you'll identify the redundant columns in the volunteer dataset, and perform feature selection on the dataset to return a DataFrame of the relevant features.

For example, if you explore the volunteer dataset in the console, you'll see three features which are related to location: locality, region, and postalcode. They contain related information, so it would make sense to keep only one of the features.

Take some time to examine the features of volunteer in the console, and try to identify the redundant features.

In [2]:
# Create a list of redundant column names to drop
to_drop = ["category_desc", "created_date", "locality", "region", "vol_requests"]

# Drop those columns from the dataset
volunteer_subset = volunteer.drop(to_drop, axis=1)

# Print out the head of volunteer_subset
print(volunteer_subset.head())
#  It's often easier to collect a list of columns to drop, rather than dropping them individually.

   opportunity_id  content_id  event_time  \
0            4996       37004           0   
1            5008       37036           0   
2            5016       37143           0   
3            5022       37237           0   
4            5055       37425           0   

                                               title  hits  \
0  Volunteers Needed For Rise Up & Stay Put! Home...   737   
1                                       Web designer    22   
2      Urban Adventures - Ice Skating at Lasker Rink    62   
3  Fight global hunger and support women farmers ...    14   
4                                      Stop 'N' Swap    31   

                                             summary is_priority  category_id  \
0  Building on successful events last summer and ...         NaN          NaN   
1             Build a website for an Afghan business         NaN          1.0   
2  Please join us and the students from Mott Hall...         NaN          1.0   
3  The Oxfam Action Corps is a g

**Checking for correlated features**   
You'll now return to the wine dataset, which consists of continuous, numerical features. Run Pearson's correlation coefficient on the dataset to determine which columns are good candidates for eliminating. Then, remove those columns from the DataFrame.

In [5]:
# Print out the column correlations of the wine dataset
print(wine.corr())

# Drop that column from the DataFrame
wine = wine.drop("Flavanoids", axis=1)

print(wine.head())

# Dropping correlated features is often an iterative process, so you may need to try different combinations in your model.

                                  Type   Alcohol  Malic acid       Ash  \
Type                          1.000000 -0.328222    0.437776 -0.049643   
Alcohol                      -0.328222  1.000000    0.094397  0.211545   
Malic acid                    0.437776  0.094397    1.000000  0.164045   
Ash                          -0.049643  0.211545    0.164045  1.000000   
Alcalinity of ash             0.517859 -0.310235    0.288500  0.443367   
Magnesium                    -0.209179  0.270798   -0.054575  0.286587   
Total phenols                -0.719163  0.289101   -0.335167  0.128980   
Flavanoids                   -0.847498  0.236815   -0.411007  0.115077   
Nonflavanoid phenols          0.489109 -0.155929    0.292977  0.186230   
Proanthocyanins              -0.499130  0.136698   -0.220746  0.009652   
Color intensity               0.265668  0.546364    0.248985  0.258887   
Hue                          -0.617369 -0.071747   -0.561296 -0.074667   
OD280/OD315 of diluted wines -0.788230

### Selecting Feature Using Text Vector

**Exploring text vectors, part 1**  
Let's expand on the text vector exploration method we just learned about, using the volunteer dataset's title tf/idf vectors. In this first part of text vector exploration, we're going to add to that function we learned about in the slides. We'll return a list of numbers with the function. In the next exercise, we'll write another function to collect the top words across all documents, extract them, and then use that list to filter down our text_tfidf vector.

- Add parameters called original_vocab, for the tfidf_vec.vocabulary_, and top_n.
- Call pd.Series() on the zipped dictionary. This will make it easier to operate on.
- Use the .sort_values() function to sort the series and slice the index up to top_n words.
- Call the function, setting original_vocab=tfidf_vec.vocabulary_, setting vector_index=8 to grab the 9th row, and setting top_n=3, to grab the top 3 weighted words.

In [11]:
# Add in the rest of the arguments
def return_weights(vectorizer, vector, vector_index, top_n):
    # Get the indices and values for the specified vector_index
    indices = vector[vector_index].indices
    data = vector[vector_index].data

    # Create a series mapping indices to data
    series = pd.Series(data, index=indices)

    # Sort the series by values and get the top n indices
    top_indices = series.sort_values(ascending=False)[:top_n].index

    # Get the feature names (words) from the vectorizer using the indices
    feature_names = vectorizer.get_feature_names_out()
    top_words = [feature_names[i] for i in top_indices]

    return top_words

# Print out the weighted words
print(return_weights(tfidf_vec, text_tfidf, 8, 3))

['cents', 'ability', 'join']


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer object
tfidf_vec = TfidfVectorizer()

# Fit and transform the title column
text_tfidf = tfidf_vec.fit_transform(volunteer['title'])

# Print the shape of the text_tfidf vector
print(text_tfidf.shape)

(665, 1136)


**Training Naive Bayes with feature selection**  
You'll now re-run the Naive Bayes text classification model that you ran at the end of Chapter 3 with our selection choices from the previous exercise: the volunteer dataset's title and category_desc columns.

In [20]:
# Drop rows where the target variable 'y' is NaN
not_nan_indices = y.dropna().index
filtered_text_cleaned = filtered_text[not_nan_indices]
y_cleaned = y.loc[not_nan_indices]

# Split the dataset according to the class distribution of category_desc
X_train, X_test, y_train, y_test = train_test_split(filtered_text_cleaned.toarray(), y_cleaned, stratify=y_cleaned, random_state=42)

# Fit the model to the training data
# Assuming 'nb' is a Naive Bayes classifier as commonly used with text data
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)

# Print out the model's accuracy
print(nb.score(X_test, y_test))

0.5612903225806452


In [17]:
# Create a list of the top 30 words across all documents
def top_tfidf_words(text_tfidf, vectorizer, top_n=30):
    all_weights = np.asarray(text_tfidf.mean(axis=0)).ravel()
    top_indices = all_weights.argsort()[-top_n:][::-1]
    # Get feature names directly from the vectorizer
    feature_names = vectorizer.get_feature_names_out()
    return feature_names[top_indices]

top_words = top_tfidf_words(text_tfidf, tfidf_vec)

# Filter the text_tfidf vector to include only the top words
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a new TfidfVectorizer with the vocabulary restricted to the top words
filtered_vectorizer = TfidfVectorizer(vocabulary=top_words)

# Fit and transform the title column again using the filtered vocabulary
filtered_text = filtered_vectorizer.fit_transform(volunteer['title'])


# Create a Series for the target variable
y = volunteer["category_desc"]

# Print the shape of the filtered_text vector
print(filtered_text.shape)

# You can see that our accuracy score wasn't that different from the score at the end of Chapter 3. But don't worry,
# this is mainly because of how small the title field is.

(665, 30)


### Dimentionality Reduction

**Using PCA**  
In this exercise, you'll apply PCA to the wine dataset, to see if you can increase the model's accuracy.

In [22]:
# Import PCA
from sklearn.decomposition import PCA

# Instantiate a PCA object
pca = PCA()

# Define the features and labels from the wine dataset
X = wine.drop("Type", axis=1)
y = wine["Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Apply PCA to the wine dataset X vector
pca_X_train = pca.fit_transform(X_train)
pca_X_test = pca.transform(X_test)

# Look at the percentage of variance explained by the different components
print(pca.explained_variance_ratio_)

[9.97802349e-01 2.02071713e-03 9.82348559e-05 5.53994004e-05
 1.10395648e-05 5.87233448e-06 3.13858204e-06 1.54420449e-06
 1.02927386e-06 3.90521513e-07 1.95535151e-07 8.99659634e-08]


**Training a model with PCA**  
Now that you have run PCA on the wine dataset, you'll finally train a KNN model using the transformed data.

In [24]:
# Instantiate a KNN model
knn = KNeighborsClassifier()

# Fit knn to the training data
knn.fit(pca_X_train, y_train)

# Score knn on the test data and print it out
print(knn.score(pca_X_test, y_test))

0.7777777777777778
